In [19]:
# import data packages
import pandas as pd
import numpy as np
import datetime
import matplotlib as plt 
from analysis import *

In [20]:
# let's make the index of betfair_data it's own column:
betfair_data.reset_index(inplace=True)
#let's rename columns in the betfair_data dataframe - SELECTION_NAME will become horses:
betfair_data.rename(columns={'SELECTION_NAME': 'Horse'}, inplace=True)
# let's rename the Event ID column in all_bets to EVENT_ID:
all_bets.rename(columns={'Event ID': 'EVENT_ID'}, inplace=True)
#let's remove the rows with NAN in Event_ID:
all_bets.dropna(subset=['EVENT_ID'], inplace=True)
betfair_data.dropna(subset=['EVENT_ID'], inplace=True)
# let's make the joining columns all the same datatypes:
all_bets['EVENT_ID'] = all_bets['EVENT_ID'].astype(int)
betfair_data['EVENT_ID'] = betfair_data['EVENT_ID'].astype(int)
all_bets['Horse'] = all_bets['Horse'].astype(str)
betfair_data['Horse'] = betfair_data['Horse'].astype(str)

In [33]:
# lets select a subset of all_bets to work with:
test_all_bets = all_bets.loc[all_bets['EVENT_ID'] == 32001967]
test_all_bets

,Horse,Type,Location,Price,Midpoint Price,EVENT_ID,Venue,Race Number,Midpoint Percentage
05-01-2023 19:07:41,Becks Monelli,RaceType.GREYHOUND_RACE,AUS,1.55,1.548889,32001967,Angle Park,4,0.999283


In [35]:
test_betfair = betfair_data.loc[betfair_data['EVENT_ID'] == 32001967]
test_betfair

,index,EVENT_ID,EVENT_DT,Horse,WIN_LOSE,BSP


In [26]:
# let's left join the betfair_data dataframe with the all_bets, using the event_ID and horse as the joining column:
data_df = all_bets.merge(betfair_data, how='left', on=['EVENT_ID', 'Horse'])
print(data_df['BSP'].unique())
data_df.head()

[nan]


,Horse,Type,Location,Price,Midpoint Price,EVENT_ID,Venue,Race Number,Midpoint Percentage,index,EVENT_DT,WIN_LOSE,BSP
0,Becks Monelli,RaceType.GREYHOUND_RACE,AUS,1.55,1.548889,32001967,Angle Park,4,0.999283,NaN,NaN,NaN,NaN
1,Americas Woman,RaceType.HORSE_RACE,USA,8.00,7.411111,32003101,Charles Town,3,0.926389,NaN,NaN,NaN,NaN
2,Ashim,RaceType.HORSE_RACE,AUS,10.00,9.890756,32001844,Stawell,6,0.989076,NaN,NaN,NaN,NaN
3,Baby Thor,RaceType.GREYHOUND_RACE,AUS,5.00,4.690909,32004101,Rockhampton,9,0.938182,NaN,NaN,NaN,NaN
4,Baptiste,RaceType.GREYHOUND_RACE,AUS,2.25,2.393750,32004267,Addington,3,1.063889,NaN,NaN,NaN,NaN


In [1]:
betfair_data.Horse.dtype

NameError: name 'betfair_data' is not defined

In [17]:

all_bets['Horse'] = all_bets['Horse'].to_string()
print(all_bets.dtypes)
print(betfair_data.dtypes)

index                   object
Horse                   object
Type                    object
Location                object
Price                  float64
Midpoint Price         float64
EVENT_ID                 int32
Venue                   object
Race Number             object
Midpoint Percentage    float64
dtype: object
EVENT_ID      int32
EVENT_DT     object
Horse        object
WIN_LOSE      int64
BSP         float64
dtype: object


In [40]:
l1 = all_bets['EVENT_ID'].unique()
l2 = betfair_data['EVENT_ID'].unique()
[i for i in l1 if i in l2]

[]

In [37]:
l1 = all_bets['Horse'].unique()
l2 = betfair_data['Horse'].unique()
[i for i in l1 if i in l2]

['Becks Monelli',
 'Americas Woman',
 'Baby Thor',
 'Bigger Than Thorn',
 'Blackpool Dill',
 'Boxing On',
 'Boy From Hill',
 'Drunken Lullaby',
 'Ellen Shannon',
 'Gee Gee True Story',
 'Komachi',
 'Mitcham Xena',
 'Next Stop',
 'October Racketeer',
 'Opawa Warrior',
 'Red Fourteen',
 'Roanne',
 'Scarlet Bling',
 'Shannonsideal',
 'Sleepy Joe',
 'Tulhurst Cuddles',
 'Way Too Soft',
 'Bend The Knee',
 'Daytona',
 'Gleefilly',
 'Lektra Dish',
 'Master Of Menace',
 'Means Business',
 'Midnight Rush',
 'Obi',
 'Orsetto',
 'Sargent Kobe',
 'Sebon Toe Less',
 'Universal Playboy']